In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
print(tf.__version__)

2.3.0


## Training a baseline model

In [2]:
root_dir = '../train_base_model/tf_datasets/flower_photos'
file_pattern = "{}/image_classification_builder-train*.tfrecord*".format(root_dir)
val_file_pattern = "{}/image_classification_builder-validation*.tfrecord*".format(root_dir)

In [3]:
file_list = tf.io.gfile.glob(file_pattern)
all_files = tf.data.Dataset.list_files( tf.io.gfile.glob(file_pattern))

val_file_list = tf.io.gfile.glob(val_file_pattern)
val_all_files = tf.data.Dataset.list_files( tf.io.gfile.glob(val_file_pattern))

train_all_ds = tf.data.TFRecordDataset(all_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)
val_all_ds = tf.data.TFRecordDataset(val_all_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)

In [4]:
def decode_and_resize(serialized_example):
    # resized image should be [224, 224, 3] and normalized to value range [0, 255] 
    # label is integer index of class.
    
    parsed_features = tf.io.parse_single_example(
    serialized_example,
    features = {
    'image/channels' :  tf.io.FixedLenFeature([], tf.int64),
    'image/class/label' :  tf.io.FixedLenFeature([], tf.int64),
    'image/class/text' : tf.io.FixedLenFeature([], tf.string),
    'image/colorspace' : tf.io.FixedLenFeature([], tf.string),
    'image/encoded' : tf.io.FixedLenFeature([], tf.string),
    'image/filename' : tf.io.FixedLenFeature([], tf.string),
    'image/format' : tf.io.FixedLenFeature([], tf.string),
    'image/height' : tf.io.FixedLenFeature([], tf.int64),
    'image/width' : tf.io.FixedLenFeature([], tf.int64)
    })
    image = tf.io.decode_jpeg(parsed_features['image/encoded'], channels=3)
    label = tf.cast(parsed_features['image/class/label'], tf.int32)
    label_txt = tf.cast(parsed_features['image/class/text'], tf.string)
    label_one_hot = tf.one_hot(label, depth = 5)
    resized_image = tf.image.resize(image, [224, 224], method='nearest')
    return resized_image, label_one_hot

def normalize(image, label):
    #Convert `image` from [0, 255] -> [0, 1.0] floats 
    image = tf.cast(image, tf.float32) / 255. + 0.5
    return image, label

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.

    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Repeat forever
    ds = ds.repeat()
    ds = ds.batch(32)
    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [5]:
# perform data engineering 
dataset = train_all_ds.map(decode_and_resize)
val_dataset = val_all_ds.map(decode_and_resize)

In [6]:
# Create dataset for training run
BATCH_SIZE = 32
VALIDATION_BATCH_SIZE = 40
dataset = dataset.map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)

val_ds = val_dataset.batch(VALIDATION_BATCH_SIZE)
    
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = prepare_for_training(dataset)

In [7]:
NUM_CLASSES = 5
IMAGE_SIZE = (224, 224)
    
train_sample_size=0
for raw_record in train_all_ds:
    train_sample_size += 1
print('TRAIN_SAMPLE_SIZE = ', train_sample_size)
validation_sample_size=0
for raw_record in val_all_ds:
    validation_sample_size += 1
print('VALIDATION_SAMPLE_SIZE = ', validation_sample_size)

STEPS_PER_EPOCHS = train_sample_size // BATCH_SIZE
VALIDATION_STEPS = validation_sample_size // VALIDATION_BATCH_SIZE

TRAIN_SAMPLE_SIZE =  3540
VALIDATION_SAMPLE_SIZE =  80


In [8]:
os.environ["TFHUB_CACHE_DIR"] = '../imagenet_resnet_v2_50_feature_vector_4'
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4",
                   trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 64, activation = 'relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'custom_class')
])

model.build([None, 224, 224, 3])

model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              23564800  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
custom_class (Dense)         (None, 5)                 325       
Total params: 23,696,261
Trainable params: 131,461
Non-trainable params: 23,564,800
_________________________________________________________________


In [10]:
decoded = val_all_ds.map(decode_and_resize)
normed = decoded.map(normalize)

In [11]:
np_img_holder = np.empty((0, 224, 224,3), float)
np_lbl_holder = np.empty((0, 5), int)
for img, lbl in normed:
    r = img.numpy() # image value extracted
    rx = np.expand_dims(r, axis=0) # expand by adding a dimension for batching images.
    lx = np.expand_dims(lbl, axis=0) # expand by adding a dimension for batching labels.
    np_img_holder = np.append(np_img_holder, rx, axis=0) # append each image to create a batch of images.
    np_lbl_holder = np.append(np_lbl_holder, lx, axis=0) # append each one-hot label to create a batch of labels.

In [12]:
np_img_holder.astype(np.float32)

array([[[[1.3039216 , 0.5352941 , 0.6215686 ],
         [1.2921569 , 0.5117647 , 0.6333333 ],
         [1.3352941 , 0.5470588 , 0.68039215],
         ...,
         [1.1117647 , 0.982353  , 1.009804  ],
         [1.0058824 , 0.8843137 , 0.9078431 ],
         [0.71960783, 0.55490196, 0.59411764]],

        [[1.2686274 , 0.5117647 , 0.57843137],
         [1.2568628 , 0.5       , 0.60588235],
         [1.3039216 , 0.53137255, 0.6490196 ],
         ...,
         [1.0647058 , 0.93529415, 0.9627451 ],
         [1.0607843 , 0.93921566, 0.9627451 ],
         [0.82941175, 0.6843137 , 0.71568626]],

        [[1.2411765 , 0.5156863 , 0.5470588 ],
         [1.2176471 , 0.50784314, 0.5745098 ],
         [1.2568628 , 0.527451  , 0.6098039 ],
         ...,
         [1.0254903 , 0.9       , 0.9196079 ],
         [0.95490193, 0.8333334 , 0.8568628 ],
         [0.88823533, 0.76666665, 0.7980392 ]],

        ...,

        [[0.5862745 , 0.64117646, 0.5392157 ],
         [0.7705883 , 0.6607843 , 0.64509803]

In [13]:
checkpoint_prefix = os.path.join('trained_resnet_vector', "train_ckpt_{epoch}")
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=os.path.join('trained_resnet_vector', 'tensorboard_logs')),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
    save_weights_only=True)]

In [14]:
model.fit(
        train_ds,
        epochs=5, 
        steps_per_epoch=STEPS_PER_EPOCHS,
        validation_data=val_ds,
        validation_steps=VALIDATION_STEPS,
        callbacks=callbacks)

Epoch 1/5
  1/110 [..............................] - ETA: 0s - loss: 1.6449 - accuracy: 0.1875WARNING:tensorflow:From /Users/mbp16/Documents/projects/tf23/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


110/110 [==============================] - 92s 832ms/step - loss: 1.2953 - accuracy: 0.6639 - val_loss: 1.1900 - val_accuracy: 0.7875
Epoch 2/5
110/110 [==============================] - 93s 846ms/step - loss: 1.1406 - accuracy: 0.8449 - val_loss: 1.1456 - val_accuracy: 0.8500
Epoch 3/5
110/110 [==============================] - 93s 841ms/step - loss: 1.1124 - accuracy: 0.8722 - val_loss: 1.1243 - val_accuracy: 0.8625
Epoch 4/5
110/110 [==============================] - 90s 818ms/step - loss: 1.0947 - accuracy: 0.8932 - val_loss: 1.1282 - val_accuracy: 0.8500
Epoch 5/5
110/110 [==============================] - 94s 859ms/step - loss: 1.0840 - accuracy: 0.9026 - val_loss: 1.1252 - val_accuracy: 0.8375


Now we have a trained model. It is not yet quantized. The integer quantization process actually requires some representative data as a reference, so possible ranges and distribution of values can be properly mapped from floating point to integer domain.
It can be any data (training, validation, or test), and the recommended size is around 100. We can use our validation data, which contains 80 samples, and it will work just as well. 
Per TFLite's representative_dataset API, we need to specify a function that is a generator to stream the representative data during the conversion process. This function is `data_generator` below:

In [18]:

def data_generator():
  for input_tensor in tf.data.Dataset.from_tensor_slices(np_img_holder.astype(np.float32)).batch(1).take(sample_size):
    yield [input_tensor]

In [22]:
sample_size = 0
for raw_record in val_all_ds:
    sample_size += 1
print('Sample size: ', sample_size)

Sample size:  80


Now we are ready to start the conversion process.

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = data_generator
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/br/nbdf88sj3cj87dvsmzhkny0h0000gn/T/tmpd_2eyi5u/assets


INFO:tensorflow:Assets written to: /var/folders/br/nbdf88sj3cj87dvsmzhkny0h0000gn/T/tmpd_2eyi5u/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_back_saved_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = data_generator
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
#converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp78tfefw7/assets


INFO:tensorflow:Assets written to: /tmp/tmp78tfefw7/assets


In [25]:
# This requires r2.3 API
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


## Saving quantized model
Once the conversion process is complete, we may save the quantized model.

In [26]:
import pathlib
root_dir = root_dir
tflite_models_dir = 'quantized_resnet_vector/tflite_int8_model'

to_save_tflite_model_dir = os.path.join(root_dir, tflite_models_dir)
saved_tflite_models_dir = pathlib.Path(to_save_tflite_model_dir) #convert string to pathlib object
saved_tflite_models_dir.mkdir(exist_ok=True, parents=True) # make directory

In [27]:
# Create a pathlib object 'tgt' to save quantized model with path and file name.
tgt = pathlib.Path(to_save_tflite_model_dir, 'converted_model_reduced.tflite')
# Write quantized model to the file.
tgt.write_bytes(tflite_model_quant)

24717552

The quantized model's size is 24 MB.

In [28]:
tgt

PosixPath('../train_base_model/tf_datasets/flower_photos/quantized_resnet_vector/tflite_int8_model/converted_model_reduced.tflite')

## Preparing test dataset from TFRecord

### Load test dataset

In [29]:
test_pattern = "{}/image_classification_builder-test.tfrecord*".format(root_dir)
test_all_files = tf.data.Dataset.list_files( tf.io.gfile.glob(test_pattern))

test_all_ds = tf.data.TFRecordDataset(test_all_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)

In [30]:
sample_size = 0
for raw_record in test_all_ds:
    sample_size += 1
print('Sample size: ', sample_size)

Sample size:  50


In [31]:
decoded = test_all_ds.map(decode_and_resize)


Convert `TFRecord` to numpy array for scoring:

In [32]:
np_img_holder = np.empty((0, 224, 224,3), float)
np_lbl_holder = np.empty((0, 5), int)
for img, lbl in decoded:
    r = img.numpy() # image value extracted
    rx = np.expand_dims(r, axis=0) # expand by adding a dimension for batching images.
    lx = np.expand_dims(lbl, axis=0) # expand by adding a dimension for batching labels.
    np_img_holder = np.append(np_img_holder, rx, axis=0) # append each image to create a batch of images.
    np_lbl_holder = np.append(np_lbl_holder, lx, axis=0) # append each one-hot label to create a batch of labels.

In [33]:
np_img_holder.astype(np.float32)

array([[[[208., 217., 160.],
         [215., 227., 155.],
         [216., 228., 144.],
         ...,
         [109., 150., 142.],
         [115., 156., 148.],
         [108., 149., 145.]],

        [[216., 226., 165.],
         [215., 225., 152.],
         [224., 236., 152.],
         ...,
         [164., 201., 209.],
         [169., 204., 210.],
         [154., 189., 195.]],

        [[210., 221., 155.],
         [213., 224., 148.],
         [220., 234., 149.],
         ...,
         [180., 213., 228.],
         [174., 211., 220.],
         [168., 205., 213.]],

        ...,

        [[ 92., 153., 112.],
         [ 68., 122.,  60.],
         [ 65., 115.,  56.],
         ...,
         [ 73., 141., 120.],
         [ 74., 144., 120.],
         [ 76., 146., 122.]],

        [[ 94., 156., 117.],
         [ 87., 143.,  82.],
         [ 55., 107.,  45.],
         ...,
         [ 82., 156., 141.],
         [ 91., 161., 150.],
         [ 87., 157., 147.]],

        [[ 95., 159., 122.],
       

Now test data is in numpy format with standardized dimension, pixel value between 0 and 255, and batched. 

### Mapping prediction to class name
I need to create a reverse lookup dictionary to map probability back to label. In other words, I want to find the index where maximum probability is positioned in the array. Map this position index to flower type. To create the lookup dictionary, I need to parse the TFRecord with feature descriptions to extract label indices and names.

In [34]:
feature_description = {
    'image/channels' :  tf.io.FixedLenFeature([], tf.int64),
    'image/class/label' :  tf.io.FixedLenFeature([], tf.int64),
    'image/class/text' : tf.io.FixedLenFeature([], tf.string),
    'image/colorspace' : tf.io.FixedLenFeature([], tf.string),
    'image/encoded' : tf.io.FixedLenFeature([], tf.string),
    'image/filename' : tf.io.FixedLenFeature([], tf.string),
    'image/format' : tf.io.FixedLenFeature([], tf.string),
    'image/height' : tf.io.FixedLenFeature([], tf.int64),
    'image/width' : tf.io.FixedLenFeature([], tf.int64)
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

parsd_ds = test_all_ds.map(_parse_function)

val_label_map = {}
# getting label mapping
for image_features in parsd_ds.take(30):
    label_idx = image_features['image/class/label'].numpy()
    label_str = image_features['image/class/text'].numpy().decode()
    if label_idx not in val_label_map:
        val_label_map[label_idx] = label_str

In [35]:
val_label_map

{4: 'tulips', 3: 'dandelion', 1: 'sunflowers', 2: 'daisy', 0: 'roses'}

In [36]:
actual = []
for i in range(len(np_lbl_holder)):
    class_key = np.argmax(np_lbl_holder[i])
    actual.append(val_label_map.get(class_key))

`actual` is a list that contains the true labels for test image in the same order as the image.

### Scoring batch images with integer quantization model

In [49]:
def lookup(np_entry, dictionary):
    class_key = np.argmax(np_entry)
    return dictionary.get(class_key)
    
def batch_predict(input_raw, input_tensor, output_tensor, dictionary):
    input_data = np.array(np.expand_dims(input_raw, axis=0), dtype=np.uint8)
    interpreter.set_tensor(input_tensor['index'], input_data)
    interpreter.invoke()
    interpreter_output = interpreter.get_tensor(output_tensor['index'])
    plain_text_label = lookup(interpreter_output, dictionary)
    return plain_text_label

### Load quantized model
Earlier, we encoded the path to the quantized model in `tgt`.

In [39]:
tgt

PosixPath('../train_base_model/tf_datasets/flower_photos/quantized_resnet_vector/tflite_int8_model/converted_model_reduced.tflite')

In [40]:
##Load the TFLite model and allocate tensors.
## tgt is the pathlib object referring to file path and <TFLITE_MODEL_NAME>.tflite
interpreter = tf.lite.Interpreter(model_path=str(tgt))
interpreter.allocate_tensors()

In [41]:
# Get input and output tensors.
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

In [42]:
input_details

{'name': 'input_1',
 'index': 270,
 'shape': array([  1, 224, 224,   3], dtype=int32),
 'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
 'dtype': numpy.uint8,
 'quantization': (0.0058823530562222, 0),
 'quantization_parameters': {'scales': array([0.00588235], dtype=float32),
  'zero_points': array([0], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [43]:
output_details

{'name': 'Identity',
 'index': 271,
 'shape': array([1, 5], dtype=int32),
 'shape_signature': array([-1,  5], dtype=int32),
 'dtype': numpy.uint8,
 'quantization': (0.00390625, 0),
 'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
  'zero_points': array([0], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [44]:
# Test the model
input_data = np.array(np.expand_dims(np_img_holder[0], axis=0), dtype=np.uint8)
interpreter.set_tensor(input_details['index'], input_data)
interpreter.invoke()

In [50]:
batch_quantized_prediction = []
for i in range(sample_size):
    plain_text_label = batch_predict(np_img_holder[i], input_details, output_details, val_label_map)
    batch_quantized_prediction.append(plain_text_label)
    print(i, plain_text_label)

0 tulips
1 dandelion
2 tulips
3 sunflowers
4 daisy
5 dandelion
6 dandelion
7 roses
8 dandelion
9 tulips
10 dandelion
11 roses
12 dandelion
13 tulips
14 roses
15 sunflowers
16 daisy
17 dandelion
18 sunflowers
19 daisy
20 roses
21 roses
22 dandelion
23 sunflowers
24 roses
25 tulips
26 tulips
27 tulips
28 dandelion
29 roses
30 roses
31 daisy
32 sunflowers
33 sunflowers
34 tulips
35 dandelion
36 daisy
37 daisy
38 roses
39 sunflowers
40 roses
41 dandelion
42 daisy
43 roses
44 sunflowers
45 daisy
46 daisy
47 roses
48 daisy
49 roses


In [51]:
batch_quantized_prediction

['tulips',
 'dandelion',
 'tulips',
 'sunflowers',
 'daisy',
 'dandelion',
 'dandelion',
 'roses',
 'dandelion',
 'tulips',
 'dandelion',
 'roses',
 'dandelion',
 'tulips',
 'roses',
 'sunflowers',
 'daisy',
 'dandelion',
 'sunflowers',
 'daisy',
 'roses',
 'roses',
 'dandelion',
 'sunflowers',
 'roses',
 'tulips',
 'tulips',
 'tulips',
 'dandelion',
 'roses',
 'roses',
 'daisy',
 'sunflowers',
 'sunflowers',
 'tulips',
 'dandelion',
 'daisy',
 'daisy',
 'roses',
 'sunflowers',
 'roses',
 'dandelion',
 'daisy',
 'roses',
 'sunflowers',
 'daisy',
 'daisy',
 'roses',
 'daisy',
 'roses']

In [52]:
actual

['tulips',
 'dandelion',
 'tulips',
 'sunflowers',
 'daisy',
 'sunflowers',
 'daisy',
 'roses',
 'dandelion',
 'tulips',
 'dandelion',
 'tulips',
 'dandelion',
 'roses',
 'roses',
 'sunflowers',
 'daisy',
 'dandelion',
 'sunflowers',
 'tulips',
 'tulips',
 'roses',
 'dandelion',
 'sunflowers',
 'roses',
 'tulips',
 'tulips',
 'tulips',
 'dandelion',
 'roses',
 'sunflowers',
 'daisy',
 'sunflowers',
 'sunflowers',
 'dandelion',
 'dandelion',
 'daisy',
 'daisy',
 'roses',
 'sunflowers',
 'roses',
 'dandelion',
 'daisy',
 'roses',
 'sunflowers',
 'daisy',
 'daisy',
 'roses',
 'daisy',
 'tulips']

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(actual, batch_quantized_prediction)

0.82

We also made a comparison between the truth and prediction. Results may vary for re-training.